# Coherence operator Ĉ

The coherence operator redistributes EPI density across resonant modes. Hermiticity guarantees real expectation values, and its spectral floor `c_min` encodes the minimal coherence compatible with structural stability.

## Operator properties

* Hermitian matrices ensure real spectra and unitary flows.
* Positive semidefinite spectra prevent ΔNFR from collapsing the node.
* Expectation values quantify the instantaneous coherence contribution of a state.

## Smoke check: spectral diagnostics

This example instantiates a modest Hermitian operator, evaluates its spectrum and measures the coherence expectation of a normalized state.

In [ ]:
import numpy as np

from tnfr.mathematics.operators import CoherenceOperator
from tnfr.mathematics.spaces import HilbertSpace

coherence_matrix = np.array(
    [
        [0.7, 0.05, 0.0],
        [0.05, 0.9, 0.02],
        [0.0, 0.02, 0.6],
    ],
    dtype=np.complex128,
)

operator = CoherenceOperator(coherence_matrix)
space = HilbertSpace(dimension=3)
state = np.array([0.6, 0.5, 0.2], dtype=np.complex128)
state = state / space.norm(state)

expectation_value = operator.expectation(state)

{
    "hermitian": bool(operator.is_hermitian()),
    "positive_semidefinite": bool(operator.is_positive_semidefinite()),
    "c_min": round(operator.c_min, 6),
    "expectation": round(expectation_value, 6),
}


## Implementation Bridge: From Theory to Code

In practical TNFR networks, the coherence operator Ĉ is approximated through the `coherence_matrix()` function, which computes structural similarities between nodes.

The matrix W computed by `coherence_matrix(G)` approximates Ĉ projected onto the computational basis:
- Each element wᵢⱼ ≈ ⟨i|Ĉ|j⟩ represents structural coherence between nodes i and j
- W combines four similarity components: phase (s_phase), EPI (s_epi), frequency (s_vf), and sense index (s_si)
- The matrix maintains Hermiticity (W = W^T) ensuring valid spectral properties

**Example: Computing coherence matrix from a TNFR network**

In [ ]:
import networkx as nx
from tnfr.metrics.coherence import coherence_matrix
from tnfr.metrics.common import compute_coherence

# Create a small TNFR network
G = nx.Graph()
G.add_node(0, theta=0.0, EPI=1.0, vf=1.0, Si=0.8)
G.add_node(1, theta=0.5, EPI=1.2, vf=1.1, Si=0.7)
G.add_node(2, theta=1.0, EPI=0.9, vf=1.2, Si=0.9)
G.add_edge(0, 1)
G.add_edge(1, 2)

# Compute coherence matrix W ≈ Ĉ
nodes, W = coherence_matrix(G, use_numpy=True)

print(f"Nodes: {nodes}")
print(f"Coherence matrix W (approximating Ĉ):")

# Convert to dense for display
if isinstance(W, list) and W and isinstance(W[0], tuple):
    # Sparse format: convert to dense
    W_dense = np.zeros((len(nodes), len(nodes)))
    for i, j, w in W:
        W_dense[i, j] = w
else:
    W_dense = np.array(W)

print(W_dense)
print(f"\nHermiticity check (W = W^T): {np.allclose(W_dense, W_dense.T)}")

# Compute eigenvalues (spectrum of Ĉ)
eigenvalues = np.linalg.eigvalsh(W_dense)
print(f"\nSpectrum σ(Ĉ): {eigenvalues}")

# Total coherence C(t) = Tr(Ĉρ)
G.nodes[0]['dnfr'] = 0.1
G.nodes[1]['dnfr'] = 0.15
G.nodes[2]['dnfr'] = 0.08
G.nodes[0]['dEPI'] = 0.05
G.nodes[1]['dEPI'] = 0.08
G.nodes[2]['dEPI'] = 0.03

C_t = compute_coherence(G)
print(f"\nTotal coherence C(t) ≈ ⟨ψ|Ĉ|ψ⟩: {C_t:.6f}")

## Interpretation

The practical coherence matrix W provides:
1. **Structural coupling weights**: wᵢⱼ values indicate how strongly nodes i and j are coherently coupled
2. **Spectral information**: Eigenvalues reveal the coherence "modes" of the network
3. **Global coherence**: C(t) aggregates network-wide structural stability

This bridges the abstract mathematical formalism (Ĉ on H_NFR) with operational network metrics, maintaining TNFR's core principle: **coherence emerges from resonant structural relationships**.

See:
- [Mathematical Foundations §3.1](../theory/mathematical_foundations.md#31-coherence-operator-ĉ) for theoretical details
- [Implementation Bridge §3.1.1](../theory/mathematical_foundations.md#311-implementation-bridge-theory-to-code) for wᵢⱼ construction
- `src/tnfr/metrics/coherence.py` for complete implementation